# Natural Language Patient Finder Bulk Execution

- Female patients > 18 years old
- Patient has diabetes mellitus
- Hypertriglyceridemia
- Patient is able to return to Johns Hopkins for follow-up appointments
- nonproliferative diabetic retinopathy
- no gestational diabetes



# Setup the Environment


In [ ]:
import requests
import json

ingress_subdomain = ""

namespace = "hackathon" # input("Enter a service namespace: ")

#Learned Intent endpoint
intentURL = "https://" + namespace  + "-learned-intent." + ingress_subdomain + "/?text="

#Key Concept Extractor endpoint
extractConceptsURL = "https://" + namespace + "-key-concept-extractor." + ingress_subdomain

#Generate CQL endpoint
generateCQLURL = "https://" + namespace + "-cql-generator." + ingress_subdomain + "/generate"

#libraries
cohortURL = "https://hackathon-ingestion-cohort-service." + ingress_subdomain + "/cohort-service/libraries"
#/libraries/nameoflibrary  (get or delete)
#/libraries (post new library)
#/libraries/nameoflibrary/patients (return list of patients)
#/libraries/nameoflibrary/patientIds (return list of patient ids)

#Further manual intervention
rmeflag = False

# Define what it means to process a fragment

In [ ]:
def processSentence(the_text):
    #the_text = input("Enter a text snippet: ")
    url = intentURL + "\"" + the_text + "\""

    resp = requests.get(url=url)
    lib_name = resp.content.decode("utf-8")
    #print("Learned intent is: ", lib_name)


    intent = lib_name
    text = the_text

    params = {'intent': intent,
           'text': text
         }
      
    concept_response = requests.get(url = extractConceptsURL, params = params)

    #print(concept_response.content)

    #Call off to generate CQL sending json from adam
    #return the new cql

    new_data = concept_response.content.decode("utf-8")
    ##print("Concepts coming back from Extractor:\n", new_data)

    data = {'conceptSet': new_data}

    cql_response = requests.post(url=generateCQLURL, data=data)

#print(cql_response.content)

    new_cql = cql_response.content.decode("utf-8")
    ##print("\nCQL coming back from Generator:\n\n", new_cql)

    if "NO CQL GENERATED" in new_cql:
        print("This fragment is understood to require further guidance-no query generated.")
        print("Please consult with medical professionals.")
        rmeflag = True
        return "RME"

    library_code = new_cql

#print(library_code)

    print("\n****CQL Library Info****")
    library_name = library_code.split('"')[1]
    print("name:", library_name)

    library_version = library_code.split("'")[1]
    print("version:", library_version)

    library_endpoint = library_name + "-" + library_version
    print("endpoint extension:", library_endpoint)

#delete a libary
    delurl = cohortURL + "/" + library_endpoint

    resp = requests.delete(delurl)
    #print(resp.content)

#     if "200" in str(resp):
#         print("Library entry deleted...")
#     else:
#         print("No current library for ", library_endpoint)
    
#post a template

    post_endpoint = cohortURL

    headers = {}
    headers["Content-Type"] = "text/plain"
    headers["Accept"] = "application/json"

    resp = requests.post(url=cohortURL, data=library_code, headers = headers)

    print("New library posted...", library_endpoint, "\n")
    #print(resp)

#run a template
    run_endpoint = cohortURL + "/" + library_endpoint + "/patientIds"
    resp = requests.get(url=run_endpoint)

    #print(resp)

    result = resp.content.decode("utf-8")

    ids = (eval(result))
    print(len(ids), "patients returned")
#     for id in ids:
#         print(id)
    
    #print('\n\n\n')

    
    run_endpoint = cohortURL + "/" + library_endpoint + "/patients"
    total_resp = requests.get(url=run_endpoint)

#print(total_resp.json())

    #print(ids)

    return(ids)

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def main(text_body):
    crits = text_body.split('\n')
#print(crits)

    print("**PROCESSING: ", crits[1])
    finalList = processSentence(crits[1])
#print("****Step result: ",finalList)

    print("Intersection so far...")
    for id in finalList:
        print("   ", id)
        
    print("\n\n\n")

    for acrit in crits[2:]:
        if len(acrit) == 0:
            continue
        print("**PROCESSING: ", acrit)
        result = processSentence(acrit)
        if result == "RME":
            rmeflag = True
            print()
        else:
        #print("****Step result: ",result)
            print("\n")
            finalList = intersection(finalList, result)
            print("Intersection so far...", len(finalList))
            for id in finalList:
                print("   ", id)
            print("\n")
    
    print("Final Result:")
    for id in finalList:
        print("   ", id)

    if rmeflag:
        print("**NOTE: Remember there was at least one fragment that requires further evaluation.")



# Provide a body of text, line by line

# Execute keeping track of patients that satisfy each line

In [ ]:
text_body = '''
patient is male or female and greater than 18 years old
bilirubin > 0.2
consent to return to hospital
patient has diabetes mellitus
'''

text_body2 = '''
Female patients > 18 years old
Patient has diabetes mellitus
Hypertriglyceridemia
Patient is able to return to Johns Hopkins for follow-up appointments
nonproliferative diabetic retinopathy
no gestational diabetes
'''

main(text_body2)